## 梯度集成算法 （Gradient Boosting）

GB 可以用于回归和分类，基于的弱分类器通常是决策树。GB 通过对可微分损失函数的优化来提高弱分类器的泛化能力。

### 思想
把 Boosting 集成看成梯度下降算法。

[Gradient Boosting](https://en.wikipedia.org/wiki/Gradient_boosting) wiki 上有详细的推导过程，介绍的很不错。

GB 的最终过是为了拟合残差。[Gradient Boosting From Scratch](https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d)。这个是很好的 Tutorial。

[机器学习算法GBDT的面试要点总结-上篇](https://www.cnblogs.com/ModifyRong/p/7744987.html)， 而这篇文章更加详细描述了应用时的内容。而且学到了 GB 所基于的 CART 树在构建时，如果是分类问题就是 Gini 系数进行划分，而如果是回归问题就用方差来划分。

### 算法实现

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

In [ ]:
class DecisionTree(object):
    def __init__(self, x, y, idxs = None, min_leaf = 2):
        '''
            构造函数：
            x, y 训练样本；
            idxs: 样本的下标, 因为每次递归划分后会产生新的样本集，而我们只需要记录新样本集的下标就行了。
            in_leaf: 最的小叶子节点数；
        '''
        if idxs is None: idxs = np.arange(len(y))
        self.x, self.y = x,y
        self.m, self.n = x.shape
        self.idxs, self.min_leaf = idxs, min_leaf
        self.val = np.mean(y[idxs])
        self.score = np.inf
    
    def find_feature_split(self):
        '''
            通过对每一个 feature 调用 find_best_split_val 找到每个 feature 的最佳分隔点。
            然后挑选最好的一个 feature.
            lhs: left-hand side
            rhs: right-hand side
        '''
        for i in range(self.n):
            self.find_best_split_val(i)
        if self.score == np.inf : return
        
        x = self.split_col
        lhs = np.nozero(x <= self.split)[0]
        rhs = np.nozero(x > self.split)[0]
        self.lhs = DecisionTree(self.x, self.y, self[lhs])
        self.rhs = DecisionTree(self.x, self.y, self[rhs])
        return
    
    def find_best_split_val(self, feature):
        x, y = self.x.values[self.idxs, feature], self.y[self.idxs]
        
        # 对于数值性特征我们使用方差来进行划分。先对特征所有的取值进行排序。
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        
        # 初始时的两分划分点会把所有样本都划分到右侧。
        lhs_cnt, lhs_sum, lhs_sum2 = 0, 0.0, 0.0
        rhs_cnt, rhs_sum, rhs_sum2 = self.m, sort_y.sum(), (sort_y ** 2).sum()
        
        for i in range(0, self.m - self.min_leaf - 1):
            xi, yi = sort_x[i], sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1;
            lhs_sum += yi; rhs_sum -= yi;
            lhs_sum2 += yi ** 2; rhs_sum2 -= yi ** 2;
            
            if i < self.min_leaf or xi == sort_x[i + 1]:
                continue
            
            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            cur_score = lhs_std * lhs_cnt + rhs_std * rhs_cnt
            if cur_score < self.score:
                self.feature, self.score, self.split_val = feature, cur_score, xi  
        return
    
    
        
        
        
    